In [1]:
!pip install tensorflow==2.12 keras==2.12 larq

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.9 MB/s eta 0:00:00
   ━━

In [1]:
import tensorflow as tf
import larq as lq
import numpy as np
import os

# Loading and preprocessing MNIST
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28 * 28).astype("float32") / 127.5 - 1
test_images = test_images.reshape(-1, 28 * 28).astype("float32") / 127.5 - 1

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
# Binarized layer parameters
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip",
    use_bias=False
)

# Defining model from the paper
model = tf.keras.models.Sequential([
    lq.layers.QuantDense(512, input_shape=(784,), **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(10, **kwargs),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("softmax")
])

In [11]:
# Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_data=(test_images, test_labels))

# Evaluating the original model
print("\n Original Model Accuracy ")
original_loss, original_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {original_acc*100:.2f}%")

Epoch 1/10
469/469 [==============================] - 22s 43ms/step - loss: 0.1506 - accuracy: 0.9561 - val_loss: 0.1468 - val_accuracy: 0.9554
Epoch 2/10
469/469 [==============================] - 20s 42ms/step - loss: 0.0792 - accuracy: 0.9772 - val_loss: 0.1384 - val_accuracy: 0.9585
Epoch 3/10
469/469 [==============================] - 19s 41ms/step - loss: 0.0671 - accuracy: 0.9806 - val_loss: 0.1553 - val_accuracy: 0.9535
Epoch 4/10
469/469 [==============================] - 18s 39ms/step - loss: 0.0592 - accuracy: 0.9823 - val_loss: 0.1422 - val_accuracy: 0.9590
Epoch 5/10
469/469 [==============================] - 19s 40ms/step - loss: 0.0514 - accuracy: 0.9843 - val_loss: 0.1425 - val_accuracy: 0.9594
Epoch 6/10
469/469 [==============================] - 18s 39ms/step - loss: 0.0528 - accuracy: 0.9836 - val_loss: 0.1202 - val_accuracy: 0.9666
Epoch 7/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0464 - accuracy: 0.9862 - val_loss: 0.1216 - val_accuracy:

In [12]:
# Function to extract and print binary weights
def get_binarized_weights(layer):
    weights = layer.get_weights()[0]
    quantizer = layer.kernel_quantizer  # Directly using kernel_quantizer of the QuantDense layer
    binarized_weights = quantizer(weights)
    return binarized_weights.numpy()

In [13]:
# Printing binary weights before column swap
print("\n Binarised Weights Before Column Swap:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        binary_weights = get_binarized_weights(layer)
        print(binary_weights[:10, :10])  # Print top-left 10x10 part of the binary weights

# Save model with binary weights
os.makedirs("model", exist_ok=True)
model.save("model/binarized_model", include_optimizer=False)
print("\n Model saved with binarised weights.")


 Binarised Weights Before Column Swap:

Layer: quant_dense
[[-1. -1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1. -1. -1.  1.  1. -1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1.]
 [ 1.  1.  1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1. -1. -1. -1. -1.]
 [ 1. -1.  1. -1.  1.  1.  1.  1.  1. -1.]
 [ 1.  1.  1. -1.  1. -1. -1. -1.  1. -1.]
 [-1. -1.  1.  1.  1. -1. -1. -1. -1.  1.]
 [-1. -1.  1. -1. -1.  1. -1.  1.  1. -1.]
 [-1. -1. -1. -1. -1.  1.  1. -1. -1.  1.]]

Layer: quant_dense_1
[[ 1. -1. -1.  1.  1.  1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1.  1.  1. -1. -1.  1.]
 [-1.  1.  1. -1. -1.  1. -1.  1. -1.  1.]
 [-1. -1. -1.  1. -1. -1.  1.  1.  1. -1.]
 [ 1.  1.  1. -1.  1. -1.  1.  1.  1.  1.]
 [ 1.  1. -1.  1. -1. -1.  1.  1.  1. -1.]
 [-1.  1.  1. -1.  1.  1. -1. -1. -1.  1.]
 [ 1.  1.  1. -1. -1. -1.  1. -1.  1. -1.]
 [-1. -1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [ 1. -1.  1. -1.  1.  1. -1. -1. -1.  1.]]

Layer: quant_dense_2
[[-1. -1. -1. -1.  1.  1.  1. -1. 


 Model saved with binarised weights.


In [14]:
# Defining PUF key
puf_key = np.random.randint(0, 2, size=256)

# Column inversion function based on PUF key
def apply_puf_column_inversion(model, puf_key):
    for i, layer in enumerate(model.layers):
        if isinstance(layer, lq.layers.QuantDense):
            weights = layer.get_weights()
            if not weights:
                continue

            W = weights[0]
            out_features = W.shape[1]

            # Creating a copy for manipulation
            W_new = np.copy(W)

            for k in range(0, out_features - 1, 2):
                if puf_key[k % len(puf_key)] == 1:
                    W_new[:, k], W_new[:, k+1] = W[:, k+1], W[:, k].copy()

            layer.set_weights([W_new])

            # Applying the same inversion to BatchNormalization layer
            if i + 1 < len(model.layers) and isinstance(model.layers[i + 1], tf.keras.layers.BatchNormalization):
                bn_weights = model.layers[i + 1].get_weights()
                if len(bn_weights) == 4:
                    gamma, beta, mean, var = bn_weights
                    for k in range(0, out_features - 1, 2):
                        if puf_key[k % len(puf_key)] == 1:
                            gamma[k], gamma[k+1] = gamma[k+1], gamma[k]
                            beta[k], beta[k+1] = beta[k+1], beta[k]
                            mean[k], mean[k+1] = mean[k+1], mean[k]
                            var[k], var[k+1] = var[k+1], var[k]
                    model.layers[i + 1].set_weights([gamma, beta, mean, var])

    print(" Column inversion applied based on PUF key.")

In [15]:
# Applying column inversion based on PUF key
apply_puf_column_inversion(model, puf_key)

# Printing weights after inversion
print("\n Binary Weights After Column Inversion:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        binary_weights = get_binarized_weights(layer)
        print(binary_weights[:10, :10])  # Print top-left 10x10 part of the binary weights

 Column inversion applied based on PUF key.

 Binary Weights After Column Inversion:

Layer: quant_dense
[[-1. -1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1.  1. -1.  1.  1. -1.  1. -1. -1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1.  1. -1. -1.]
 [ 1.  1.  1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1.  1. -1. -1. -1. -1. -1.]
 [-1.  1. -1.  1.  1.  1.  1.  1.  1. -1.]
 [ 1.  1. -1.  1. -1.  1. -1. -1.  1. -1.]
 [-1. -1.  1.  1. -1.  1. -1. -1. -1.  1.]
 [-1. -1. -1.  1.  1. -1. -1.  1.  1. -1.]
 [-1. -1. -1. -1.  1. -1.  1. -1. -1.  1.]]

Layer: quant_dense_1
[[-1.  1.  1. -1.  1.  1. -1. -1.  1.  1.]
 [-1. -1. -1. -1.  1. -1.  1. -1. -1.  1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1. -1.  1.]
 [-1. -1.  1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1.  1.  1.  1. -1.]
 [ 1. -1. -1.  1.  1.  1. -1. -1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1.  1. -1.  1. -1.]
 [-1. -1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [-1.  1. -1.  1.  1.  1. -1. -1. -1.  1.]]

Layer: qua

In [16]:
# Evaluating the model after column inversion
print("\n Swapped Model Accuracy ")
swapped_loss, swapped_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy After Column Inversion: {swapped_acc*100:.2f}%")


 Swapped Model Accuracy 
313/313 [==============================] - 3s 8ms/step - loss: 2.0958 - accuracy: 0.4203
Test Accuracy After Column Inversion: 42.03%


In [17]:
# Saving the model with the column inversion applied
os.makedirs("model_swapped", exist_ok=True)
model.save("model_swapped/binarized_model_swapped", include_optimizer=False)
print("\n Model saved after column inversion.")


 Model saved after column inversion.


In [18]:
# Function to print the size of the weight matrix for each QuantDense layer
def print_weight_shapes(model):
    for i, layer in enumerate(model.layers):
        if isinstance(layer, lq.layers.QuantDense):
            weights = layer.get_weights()
            if weights:
                print(f"Layer {i + 1} weight matrix shape: {weights[0].shape}")

# Printing the weight matrix shapes
print_weight_shapes(model)

Layer 1 weight matrix shape: (784, 512)
Layer 3 weight matrix shape: (512, 512)
Layer 5 weight matrix shape: (512, 512)
Layer 7 weight matrix shape: (512, 10)


In [ ]:
# For load the model later, the following code should be used:
from larq.layers import QuantDense
from larq.quantizers import ste_sign
from larq.constraints import WeightClip

custom_objects = {
    "QuantDense": QuantDense,
    "ste_sign": ste_sign,
    "WeightClip": WeightClip
}

# Loading model with quantizers intact
model = tf.keras.models.load_model("model_swapped/binarized_model_swapped", custom_objects=custom_objects)